In [ ]:
import sys
import torch
import torchtext
!pip3 install fastai
import fastai
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import pandas as pd
import torchtext
from fastai.text import *


In [ ]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

data_path =  '/content/gdrive/My Drive/APS360/APS360: Team Project/Project/Project Data' 
hate_tweets_path = data_path + '/hatespeech' + "/stage2tweets.csv"
p2p_path = data_path + '/hatespeech' + '/p2p'+'/tweets.csv'

# Hatespeech Data
data_hatespeech_1 = pd.read_csv(hate_tweets_path)
data_hatespeech_2 = pd.read_csv(p2p_path)

Mounted at /content/gdrive


In [ ]:
def check_balance(ds):
  num_neg = 0
  num_pos=0
  for example in ds :
  #print(str(example.label)+", "+str(example.tweet))
    if example.label == 1:
      num_pos +=1
    elif example.label == 0:
      num_neg +=1
  print("neg:" +str(num_neg)+" pos:" +str(num_pos))

In [ ]:
data_hatespeech_2 = data_hatespeech_2.transpose()
data_hatespeech_2.columns = ['tweet']

length = data_hatespeech_2.shape[0]
data_hatespeech_2['hate_speech'] = np.ones(length)

data_hatespeech_2 = data_hatespeech_2.reset_index()
data_hatespeech_2.head()

,index,tweet,hate_speech
0,0,@AounJon if you can't admit it's a positive no...,1.0
1,1,@RURALINDIA @bainjal oh wow wen we hit hard in...,1.0
2,2,@DiaQuery @NameFieldmt @YearOfRat They shed le...,1.0
3,3,@vivelafra @LindaSuhler This is retarded. Just...,1.0
4,4,@_MrSp33dy_ @BTSpill @Pawmso @Average_AJ you k...,1.0


In [ ]:
# Modify Hatespeech 1
hatespeech1 = data_hatespeech_1.drop(['Unnamed: 0','count','neither','class'],axis='columns')
for i in range(hatespeech1.shape[0]):
  if hatespeech1.loc[i,"hate_speech"] > 0:
    hatespeech1.loc[i,"hate_speech"] = 1
  if hatespeech1.loc[i,"offensive_language"] > 3: # If the offensive language column reads greater than 3, we're classifying as hate speech (can discuss this)
    hatespeech1.loc[i,"hate_speech"] = 1
hatespeech1 = hatespeech1.drop(['offensive_language'],axis='columns')
hatespeech1 = hatespeech1.rename(columns={"hate_speech": "label"})
print(hatespeech1.head(10))

# Modify Hatespeech 2
data_hatespeech2_idx = data_hatespeech_2.rename(columns={"hate_speech": "label"})
data_hatespeech2 = data_hatespeech2_idx.drop(['index'],axis='columns')
print(data_hatespeech2.head(10))

# Concatenate the two hatespeech datasets:
hatespeech_final = pd.concat([hatespeech1, data_hatespeech2],ignore_index=True)


   label                                              tweet
0      0  !!! RT @mayasolovely: As a woman you shouldn't...
1      0  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      0  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      0  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5      1  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6      0  !!!!!!"@__BrighterDays: I can not just sit up ...
7      0  !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8      0  " &amp; you might not get ya bitch back &amp; ...
9      1  " @rhythmixx_ :hobbies include: fighting Maria...
                                               tweet  label
0  @AounJon if you can't admit it's a positive no...    1.0
1  @RURALINDIA @bainjal oh wow wen we hit hard in...    1.0
2  @DiaQuery @NameFieldmt @YearOfRat They shed le...    1.0
3  @vivelafra @LindaSuhler This is retarded. Just...    1.0
4  @_MrSp33dy_ @BTSpill @Pawmso @Average

In [ ]:
hatespeech_final.to_csv(data_path + '/hatespeech/combined_hatespeech_data.csv', index=False)